In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from PIL import Image
import glob
import time
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Define paths
# data_dir = '/content/gdrive/MyDrive/sickle3-edofed/'
data_dir = 'C:/Users/billy/Documents/PythonScripts/converted_cells_dataset'

In [ ]:
# Training parameters
epochs = 50
batch_size = 16
image_size = 64
latent_dim = 128
gp_weight = 10.0

In [ ]:
# input_dir = 'C:/Users/billy/Documents/PythonScripts/cells_dataset'
# output_dir = 'C:/Users/billy/Documents/PythonScripts/converted_cells_dataset'

# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# for subdir, dirs, files in os.walk(input_dir):
#     for file in files:
#         if file.lower().endswith('.tiff'):
#             input_path = os.path.join(subdir, file)
#             output_subdir = subdir.replace(input_dir, output_dir)
#             os.makedirs(output_subdir, exist_ok=True)
#             output_path = os.path.join(output_subdir, file[:-5] + '.png')

#             with Image.open(input_path) as img:
#                 img.save(output_path)


In [ ]:
# Load dataset
def load_images(data_dir, image_size, batch_size):
    dataset = image_dataset_from_directory(
        data_dir,
        label_mode=None,
        color_mode='rgb',
        batch_size=batch_size,
        image_size=(image_size, image_size),
        shuffle=True,
        seed=42,
    )
    dataset = dataset.map(lambda x: x / 255.0)
    return dataset

In [ ]:
# Create the generator
def create_generator(latent_dim, channels):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(latent_dim,)))
    model.add(layers.Dense(4 * 4 * 512))
    model.add(layers.Reshape((4, 4, 512)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(32, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(channels, kernel_size=5, strides=1, padding='same', activation='tanh'))
    return model

In [ ]:
# Create the discriminator
def create_discriminator(image_shape):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=image_shape))

    model.add(layers.Conv2D(32, kernel_size=5, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(64, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(128, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Conv2D(256, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

In [ ]:
# Define loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

def discriminator_loss(real_output, fake_output):
    real_loss = tf.reduce_mean(tf.nn.relu(1.0 - real_output))
    fake_loss = tf.reduce_mean(tf.nn.relu(1.0 + fake_output))
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# Define gradient
@tf.function
def gradient_penalty(real_images, fake_images, discriminator):
    alpha = tf.random.normal([real_images.shape[0], 1, 1, 1], 0.0, 1.0)
    interpolates = alpha * real_images + (1 - alpha) * fake_images
    with tf.GradientTape() as tape:
        tape.watch(interpolates)
        d_interpolates = discriminator(interpolates)
    gradients = tape.gradient(d_interpolates, [interpolates])[0]
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]))
    gradient_penalty = tf.reduce_mean((slopes - 1.0) ** 2)
    return gradient_penalty

In [ ]:
# Define the training loop
@tf.function
def train_step(images, generator, discriminator, generator_optimizer, discriminator_optimizer):
    noise = tf.random.normal([images.shape[0], latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = -tf.reduce_mean(fake_output)
        disc_loss = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

        gp = gradient_penalty(images, generated_images, discriminator)
        disc_loss += gp * gp_weight

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Generate images
    predictions = model(test_input, training=False)

    # Create a directory for generated images if it doesn't exist
    output_dir = 'C:/Users/billy/Documents/PythonScripts/GAN_images'
    os.makedirs(output_dir, exist_ok=True)

    # Save the images
    for i in range(predictions.shape[0]):
        image_array = (predictions[i].numpy() * 127.5 + 127.5).astype(np.uint8)
        img = Image.fromarray(image_array)
        img.save(os.path.join(output_dir, f'image_at_epoch_{epoch:04d}_sample_{i}.png'))

    print(f'Saved generated images for epoch {epoch}')

In [ ]:
def train(dataset, epochs, generator, discriminator, generator_optimizer, discriminator_optimizer):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        for i, image_batch in enumerate(dataset):
            gen_loss, disc_loss = train_step(image_batch, generator, discriminator, generator_optimizer, discriminator_optimizer)
            print(f"Batch {i + 1}: gen_loss={gen_loss.numpy()}, disc_loss={disc_loss.numpy()}")

        if (epoch + 1) % 5 == 0:
            generator.save(f"generator_epoch_{epoch + 1}.h5")

In [ ]:
generator = create_generator(latent_dim, 3)
discriminator = create_discriminator((image_size, image_size, 3))

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

dataset = load_images(data_dir, image_size, batch_size)
print(f"Loaded {len(glob.glob(os.path.join(data_dir, '*/*')))} images")

train(dataset, epochs, generator, discriminator, generator_optimizer, discriminator_optimizer)

In [ ]:
if __name__ == '__main__':
    # Load the dataset
    images = load_images(data_dir)
    print(f'Loaded {len(images)} images')

    # Create a TensorFlow dataset
    noise_dim = 100
    num_examples_to_generate = 16
    seed = tf.random.normal([num_examples_to_generate, noise_dim])
    dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(len(images)).batch(batch_size)

    # Create generator and discriminator
    generator = create_generator()
    discriminator = create_discriminator()

    # Train the GAN
    train(dataset, epochs)

    # Save the generator model
    generator.save('/content/gdrive/MyDrive/gan_generator.h5')

In [ ]:
# Generate Images
def generate_and_save_images(model, epoch, test_input, output_dir):
    # Generate images using the test_input
    predictions = model(test_input, training=False)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Plot and save the generated images
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i] + 1) / 2)
        plt.axis('off')

    plt.savefig(os.path.join(output_dir, f'image_at_epoch_{epoch:04d}.png'))
    plt.show()

In [ ]:
# Number of images to generate (e.g., 16)
num_examples_to_generate = 8

# Generate random noise as input for the generator
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
generate_and_save_images(generator, epoch=epochs, test_input=seed, output_dir='generated_images')